<a href="https://colab.research.google.com/github/neel26desai/advanced_prompt_engineering_stategies/blob/main/Prompt_Engineering_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain langchain-community
!pip install langchain-openai
!pip install langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
from google.colab import userdata
import os

In [3]:
import openai
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, api_key = userdata.get('open_ai'))

In [ ]:
messages = [
    ("human", "Hello"),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-f585ea1a-6f23-4e8e-98b1-a14dbde1b75f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
ai_msg.content

'Hello! How can I assist you today?'

In [5]:
def talk(llm,msg):
  messages.append(("human", msg))
  ai_msg = llm.invoke(messages)
  messages.append(("ai", ai_msg.content))
  return ai_msg.content

## In Context Learning (ICL)

ICL involves giving GPT examples within the prompt to allow it to learn on the fly.

In [ ]:
#without ICL, showing limitation
prompt = "Who won the 2022 World Cup?"
response = talk(llm,prompt)
print(response)


Argentina won the 2022 FIFA World Cup. They defeated France in the final, which took place on December 18, 2022, in Qatar. The match ended in a dramatic 3-3 draw after extra time, and Argentina triumphed 4-2 in the penalty shootout. This victory marked Argentina's third World Cup title, with their previous wins in 1978 and 1986.


The default prompt gives us too much information and it is no in the form that we want.

So we use incontext learning to teach the LLM on what information do we need and how exactly we need it.

In [ ]:
#using ICL to address the limitation
prompt='''Answer the question using the format
shown in the context.
Question: Who won the World Cup in 2014?
Answer: Germany won the 2014 FIFA World Cup. They defeated Argentina in the final, leadin 1 goal to 0.


Who won the world cup in 2022?'''
response = talk(llm,prompt)
print(response)

Question: Who won the World Cup in 2022?  
Answer: Argentina won the 2022 FIFA World Cup. They defeated France in the final, leading 3 goals to 2 after a penalty shootout.


## Chain of Thought

Chain-of-Thought (CoT) Prompting is a technique that enhances the reasoning capabilities of large language models (LLMs) by incorporating logical steps—or a “chain of thought”—within the prompt.

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key = userdata.get('open_ai'))

Suppose you have a receipe for making coffee, in spanish, and you want to know all the coffee related words in the receiped in English.

In [ ]:
#failure case
input_text = """
¡Preparar café Cold Brew es un proceso sencillo y refrescante!
Todo lo que necesitas son granos de café molido grueso y agua fría.
Comienza añadiendo el café molido a un recipiente o jarra grande.
Luego, vierte agua fría, asegurándote de que todos los granos de café
estén completamente sumergidos.
Remueve la mezcla suavemente para garantizar una saturación uniforme.
Cubre el recipiente y déjalo en remojo en el refrigerador durante al
menos 12 a 24 horas, dependiendo de la fuerza deseada.
"""
prompt = f"""
Give me a numbered list of all coffee-related words in English from the text below:
Text: <{input_text}>
"""
response = talk(llm,prompt)
print(response)

1. café
2. Cold Brew
3. granos de café
4. café molido
5. agua fría
6. recipiente
7. jarra
8. agua fría
9. granos de café
10. mezcla
11. saturación
12. refrigerador
13. fuerza


we can see that although we got a list of all coffee related words, they are still in spanish.

In [ ]:
#overcoming limitation
prompt = f"""
Give me a numbered list of all coffee-related words in English from the text below:
The task requires the following actions:
1 - Translate the given text into English.
2 - List each coffee-related word from the English text.
Text: <{input_text}>
"""
response = talk(llm,prompt)
print(response)

1. coffee
2. Cold Brew
3. coffee beans
4. ground coffee
5. cold water
6. container
7. jar
8. water
9. coffee beans
10. mixture
11. saturation
12. refrigerator
13. strength


By telling the model exactly the steps in needs to performs, allows us to get the current answer.

## iCOT

We can impove the on CoT prompting, by using iterative CoT.

In [ ]:
prompt = f"""
Give me a numbered list of all coffee-related words in English from the text below:
Text: <{input_text}>
Lets think step by step.
First I want you to find all the steps required for solving the problem.
"""
first_response = talk(llm,prompt)
print('Printing the first Prompt')
print(first_response)

prompt += f"\n{first_response}\n Run and the steps you identified and give the answer"

second_response = talk(llm,prompt)
print('Printig the second Prompt')
print(second_response)



Printing the first Prompt
1. Translate the text into English.
2. Identify all coffee-related words in the English text.
3. List the coffee-related words in English.
Printig the second Prompt
1. Translate the text into English:
Prepare Cold Brew coffee is a simple and refreshing process! All you need are coarse ground coffee beans and cold water. Start by adding the ground coffee to a large container or jar. Then, pour cold water, making sure all the coffee beans are completely submerged. Gently stir the mixture to ensure even saturation. Cover the container and let it soak in the refrigerator for at least 12 to 24 hours, depending on the desired strength.

2. Identify all coffee-related words in the English text:
- coffee
- Cold Brew
- ground coffee beans
- ground coffee
- coffee beans
- saturation

3. List the coffee-related words in English:
1. coffee
2. Cold Brew
3. ground coffee beans
4. ground coffee
5. coffee beans
6. saturation

These are the coffee-related words found in the te

Using iCOT we got more preciese words that are directly relaed to coffee.

## Tree of Thought (TOT

In the ToT (Tree of Thoughts) approach, LLMs evaluate themselves at each stage of thought and stop inefficient approaches early, switching to alternative methods.)

In [ ]:
from langchain.chains import LLMChain

In [ ]:
api_key = userdata.get('open_ai')

In [ ]:
template ="""
Step1 :

I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template
)

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, api_key=api_key ,model="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="solutions"
)
template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, api_key=api_key, model="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="review"
)

template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, api_key=api_key, model="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="deepen_thought_process"
)

template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, api_key=api_key, model="gpt-3.5-turbo"),
    prompt=prompt,
    output_key="ranked_solutions"
)

<ipython-input-61-84277b572043>:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(


In [ ]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

response=overall_chain({"input":"human colonization of Mars", "perfect_factors":"The distance between Earth and Mars is very large, making regular resupply difficult"})



> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
print(response)

{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Based on the evaluations and scenarios provided, the ranking of the solutions in order of promise would be as follows:\n\n1. Develop sustainable habitats on Mars:\n- Justification: This solution offers the potential for long-term human habitation on Mars, addressing the fundamental need for shelter and sustainability. By partnering with private space companies and international space agencies, as well as leveraging expertise in habitat development, this solution has the potential to create a viable living environment on Mars.\n- Final thoughts: Overcoming the obstacles of creating self-sufficient ecosystems and securing funding will be challenging, but the long-term benefits of establishing sustainable habitats on Mars make this solution highly promising.\n\n2. Regular cargo missions to Mars:\n- Justification: This solut

In [ ]:
response

{'input': 'human colonization of Mars',
 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult',
 'ranked_solutions': 'Based on the evaluations and scenarios provided, the ranking of the solutions in order of promise would be as follows:\n\n1. Develop sustainable habitats on Mars:\n- Justification: This solution offers the potential for long-term human habitation on Mars, addressing the fundamental need for shelter and sustainability. By partnering with private space companies and international space agencies, as well as leveraging expertise in habitat development, this solution has the potential to create a viable living environment on Mars.\n- Final thoughts: Overcoming the obstacles of creating self-sufficient ecosystems and securing funding will be challenging, but the long-term benefits of establishing sustainable habitats on Mars make this solution highly promising.\n\n2. Regular cargo missions to Mars:\n- Justification: This sol

## Algorithm Of Thoughts (AOT)

In [12]:
#without out
prompt = "Data centers are increasingly crucial for supporting global digital infrastructure but also contribute significantly to energy consumption. What are the environmental implications of this increased data center usage?"
response = llm.invoke(prompt)
print(response.content)

The increasing reliance on data centers to support global digital infrastructure has several environmental implications, primarily related to energy consumption, resource use, and waste generation. Here are some key points to consider:

1. **Energy Consumption**: Data centers consume a substantial amount of electricity, often accounting for a significant portion of a region's total energy use. This high demand can lead to increased greenhouse gas emissions, especially if the energy is sourced from fossil fuels. The growth of cloud computing and digital services is expected to further escalate energy consumption.

2. **Carbon Footprint**: The carbon footprint of data centers is a major concern. As more data centers are built and existing ones are expanded, their collective carbon emissions can contribute to climate change. Efforts to transition to renewable energy sources are critical to mitigating this impact, but the pace of adoption varies widely.

3. **Cooling Requirements**: Data c

In [10]:
#using AOT
prompt = '''
Problem Statement:
Data centers are increasingly crucial for supporting global digital infrastructure but also contribute significantly to energy consumption. What are the environmental implications of this increased data center usage?
Yours response to include the following points and their consideratiosns
1. Background Information:
Consider that data centers currently account for about 1% of global electricity use, with projections indicating potential growth as digital demand increases.
2. Initial Hypothesis:
One potential solution to mitigate the environmental impact of data centers is the implementation of renewable energy sources. This could reduce their carbon footprint and reliance on non-renewable energy.
3. Reasoning:
Evaluate the current energy consumption patterns of data centers.
Analyze the feasibility and impact of integrating renewable energy sources, such as solar or wind power, into data center operations.
Consider technological, economic, and logistical challenges that might affect this transition.
4. Conclusion:
Based on your analysis, determine whether adopting renewable energy is a viable and effective strategy for reducing the environmental impact of data centers. Discuss any additional measures that could complement this approach. This prompt guides the model through a structured reasoning process, encouraging it to explore each step thoroughly before arriving at a conclusion.'''
response = llm.invoke(prompt)
print(response)

content='### Background Information\n\nData centers are essential for the functioning of modern digital infrastructure, supporting everything from cloud computing to streaming services. Currently, they account for approximately 1% of global electricity consumption, a figure that is expected to rise as the demand for digital services continues to grow. This increase in energy consumption raises significant environmental concerns, particularly regarding greenhouse gas emissions and the overall carbon footprint associated with energy production. As data centers expand, their energy requirements could lead to greater reliance on fossil fuels, exacerbating climate change and environmental degradation.\n\n### Initial Hypothesis\n\nTo mitigate the environmental impact of data centers, one promising solution is the implementation of renewable energy sources. By transitioning to renewable energy, such as solar, wind, or hydroelectric power, data centers could significantly reduce their carbon e

In [11]:
print(response.content)

### Background Information

Data centers are essential for the functioning of modern digital infrastructure, supporting everything from cloud computing to streaming services. Currently, they account for approximately 1% of global electricity consumption, a figure that is expected to rise as the demand for digital services continues to grow. This increase in energy consumption raises significant environmental concerns, particularly regarding greenhouse gas emissions and the overall carbon footprint associated with energy production. As data centers expand, their energy requirements could lead to greater reliance on fossil fuels, exacerbating climate change and environmental degradation.

### Initial Hypothesis

To mitigate the environmental impact of data centers, one promising solution is the implementation of renewable energy sources. By transitioning to renewable energy, such as solar, wind, or hydroelectric power, data centers could significantly reduce their carbon emissions and re

## RASCEF

In [13]:
prompt = '''
Role:
You are an expert environmental analyst specializing in sustainable technologies.
Action:
Generate a comprehensive report on renewable energy solutions for data centers.
Steps:
Identify the current energy consumption patterns of data centers.
Analyze the potential of various renewable energy sources, such as solar, wind, and hydroelectric power, for integration into data center operations.
Evaluate the benefits and challenges associated with each renewable energy source.
Provide recommendations for implementing these solutions effectively.
Context:
Data centers are significant consumers of electricity and contribute to global carbon emissions. Transitioning to renewable energy sources can potentially reduce their environmental impact.
Example:
An example section might include a comparison of solar and wind energy, detailing installation costs, energy output, and geographic considerations.
Format:
The report should be structured with an introduction, detailed analysis sections for each renewable energy source, a comparison table, and a conclusion with recommendations. Use headings and bullet points where appropriate for clarity. '''
response = llm.invoke(prompt)
print(response.content)

# Comprehensive Report on Renewable Energy Solutions for Data Centers

## Introduction

Data centers are critical components of the digital economy, providing the infrastructure necessary for cloud computing, data storage, and online services. However, they are also significant consumers of electricity, accounting for approximately 1-2% of global energy consumption and contributing to carbon emissions. Transitioning to renewable energy sources is essential for reducing the environmental impact of data centers. This report explores various renewable energy solutions, including solar, wind, and hydroelectric power, and provides recommendations for their effective implementation.

## Current Energy Consumption Patterns of Data Centers

- **Energy Demand**: Data centers require substantial energy for computing, cooling, and supporting infrastructure. The average data center consumes between 1 to 2 megawatts of power.
- **Growth Trends**: With the increasing demand for cloud services and da

## ReAct

In [14]:
prompt = '''
Task: Optimize the energy efficiency of a data center using renewable energy sources.
Reasoning:
Identify Problem Areas: Analyze current energy consumption patterns to identify inefficiencies.
Explore Solutions: Consider various renewable energy sources (e.g., solar, wind) and their potential integration into the data center's power supply.
Evaluate Feasibility: Assess the technological and economic feasibility of these solutions.
Acting:
Data Collection: Retrieve data on current energy usage and costs associated with different renewable technologies.
Simulation: Run simulations to predict the impact of integrating renewable sources on overall energy efficiency.
Implementation Plan: Develop a step-by-step implementation plan based on simulation results.
Observation and Adjustment:
Monitor changes in energy consumption post-implementation.
Adjust strategies based on observed outcomes to enhance efficiency further.'''
response = llm.invoke(prompt)
print(response.content)

### Optimizing Energy Efficiency of a Data Center Using Renewable Energy Sources

#### Step 1: Identify Problem Areas

1. **Data Collection**: 
   - Gather historical data on energy consumption, peak usage times, and operational costs.
   - Identify the major energy-consuming components (servers, cooling systems, lighting, etc.).

2. **Energy Audit**: 
   - Conduct a thorough energy audit to pinpoint inefficiencies, such as outdated equipment, poor insulation, or inefficient cooling methods.
   - Analyze the Power Usage Effectiveness (PUE) to understand how much energy is used by the data center compared to the energy used by the IT equipment alone.

#### Step 2: Explore Solutions

1. **Renewable Energy Sources**:
   - **Solar Energy**: Evaluate the feasibility of installing solar panels on the data center roof or nearby land. Consider factors like sunlight exposure, local regulations, and potential energy output.
   - **Wind Energy**: Assess the potential for wind turbines if the loca

## Graph of thought (GOT)

In [ ]:
!pip install graph_of_thoughts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.1 MB/s eta 0:00:00


In [ ]:
# Step 1: Define the problem
problem_statement = "A classroom has two blue chairs for every three red chairs. If there are 30 chairs in total, how many blue chairs are there?"

# Step 2: Create a LangChain prompt template for reasoning
reasoning_prompt = PromptTemplate(
    input_variables=["step"],
    template="Step {step}: Let's analyze the problem and solve it step by step."
)

In [ ]:
!pip install openai==0.28.0

In [ ]:
import openai
openai.__version__

'1.54.4'

In [ ]:
from graph_of_thoughts import controller, language_models, operations

In [ ]:
gop = operations.GraphOfOperations()
gop.append_operation(operations.Generate())  # Generate intermediate steps
gop.append_operation(operations.Score(scoring_function=lambda x: len(x)))  # Example scoring function
gop.append_operation(operations.GroundTruth(lambda x: x == "12"))  # Validate final answer